In [1]:
import pandas as pd
from sklearn import preprocessing
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('Data/DANE_Dengue_Data_Variables_v2.csv', encoding='latin1')
data

,Statecode,State,Municipalitycode,Municipality,Population2007,Population2008,Population2009,Population2010,Population2011,Population2012,...,2019/w43,2019/w44,2019/w45,2019/w46,2019/w47,2019/w48,2019/w49,2019/w50,2019/w51,2019/w52
0,5,Antioquia,5001,Medellín,2286126,2309689,2331389,2353410,2379920,2410046,...,29,17,35,16,21,15,20,30,14,5
1,5,Antioquia,5002,Abejorral,20643,20944,21197,21355,20973,20067,...,0,0,0,0,0,0,0,0,0,0
2,5,Antioquia,5004,Abriaquí,2261,2290,2321,2353,2386,2424,...,0,0,0,0,0,0,0,0,0,0
3,5,Antioquia,5021,Alejandría,3935,4020,4096,4171,4244,4337,...,0,0,0,0,0,0,0,0,0,0
4,5,Antioquia,5030,Amagá,27996,28331,28624,28839,28776,28371,...,0,0,0,0,0,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1117,97,Vaupés,97889,Yavaraté,555,693,841,1003,1164,1275,...,0,0,0,0,0,0,0,0,0,0
1118,99,Vichada,99001,PuertoCarreño,13742,14559,15346,16124,16934,17773,...,0,1,2,1,1,0,0,0,1,0
1119,99,Vichada,99524,LaPrimavera,5190,5784,6365,6920,7496,8034,...,1,1,1,0,0,0,0,0,0,0
1120,99,Vichada,99624,SantaRosalía,3167,3378,3586,3846,4067,4325,...,0,0,0,0,0,0,0,0,0,0


# Temperature, Percipitation and Dengue Cases weekly 2007-2018 for Medellin

# Data Pre-Processing

Temperature, Percipitation, Cases for Medellin from original data  (sivigila - cases) (google earth engine - precipitation and worldclim - temperature)

In [3]:
# Take just the row of Medellin from original dataframe
data_medellin = data[data.Municipalitycode==5001]
data_medellin

,Statecode,State,Municipalitycode,Municipality,Population2007,Population2008,Population2009,Population2010,Population2011,Population2012,...,2019/w43,2019/w44,2019/w45,2019/w46,2019/w47,2019/w48,2019/w49,2019/w50,2019/w51,2019/w52
0,5,Antioquia,5001,Medellín,2286126,2309689,2331389,2353410,2379920,2410046,...,29,17,35,16,21,15,20,30,14,5


## Temperature

In [4]:
from datetime import timedelta

temperature_medellin = pd.read_csv('Data/new_weekly_temperature.csv')[['Date', 'temperature']]
temperature_medellin.rename(columns={'Date':'LastDayWeek'}, inplace=True)
temperature_medellin.LastDayWeek = pd.to_datetime(temperature_medellin.LastDayWeek)
temperature_medellin.LastDayWeek = temperature_medellin.LastDayWeek + timedelta(days=6)
temperature_medellin

,LastDayWeek,temperature
0,2007-01-06,19.615357
1,2007-01-13,22.234000
2,2007-01-20,22.268857
3,2007-01-27,23.546000
4,2007-02-03,25.353071
...,...,...
674,2019-12-07,23.005214
675,2019-12-14,23.116214
676,2019-12-21,21.851143
677,2019-12-28,23.223143


# Precipitation

In [5]:
precipitation_medellin = pd.read_excel('Data/Data_precipitation_chirps (1).xlsx')
precipitation_medellin.rename(columns={'Unnamed: 0':'LastDayWeek','Medellin':'percipitation_medellin', 'MedellinRural':'percipitation_medellin_rural','MedellinUrbano':'percipitation_medellin_urbano'}, inplace=True)
precipitation_medellin.LastDayWeek = pd.to_datetime(precipitation_medellin.LastDayWeek)
precipitation_medellin.LastDayWeek = precipitation_medellin.LastDayWeek + timedelta(days=6)
precipitation_medellin

,LastDayWeek,percipitation_medellin,percipitation_medellin_rural,percipitation_medellin_urbano
0,2007-01-06,10.400423,11.932660,9.624380
1,2007-01-13,19.510615,20.182476,19.153142
2,2007-01-20,2.036514,0.825607,2.551441
3,2007-01-27,19.405491,21.616968,18.439284
4,2007-02-03,0.000000,0.000000,0.000000
...,...,...,...,...
673,2019-11-30,15.607313,15.862694,15.462729
674,2019-12-07,9.035788,6.591552,10.151920
675,2019-12-14,26.450263,26.978917,26.226603
676,2019-12-21,13.327096,14.847955,12.606804


## Dengue Cases

In [6]:
# Take columns of Dengue cases from Medellin data
x_cases_medellin = data_medellin[data_medellin.columns[55:]]
# convert to Row
x_cases_transposed_medellin = x_cases_medellin.T
# Rename Column
x_cases_transposed_medellin = x_cases_transposed_medellin.rename(columns= {0:'cases_medellin'})  ##integer
x_cases_transposed_medellin

,cases_medellin
2007/w01,1
2007/w02,0
2007/w03,0
2007/w04,0
2007/w05,0
...,...
2019/w48,15
2019/w49,20
2019/w50,30
2019/w51,14


## Date of Dengue Cases

In [7]:
# Create new Column
x_cases_transposed_medellin['DATA'] = x_cases_transposed_medellin.index.str.split('/')  #index column split function though works on string only -> yyyy week
x_cases_transposed_medellin

,cases_medellin,DATA
2007/w01,1,"[2007, w01]"
2007/w02,0,"[2007, w02]"
2007/w03,0,"[2007, w03]"
2007/w04,0,"[2007, w04]"
2007/w05,0,"[2007, w05]"
...,...,...
2019/w48,15,"[2019, w48]"
2019/w49,20,"[2019, w49]"
2019/w50,30,"[2019, w50]"
2019/w51,14,"[2019, w51]"


In [8]:
# Take year
x_cases_transposed_medellin['YEAR'] = x_cases_transposed_medellin['DATA'].apply(lambda x: x[0])
x_cases_transposed_medellin

,cases_medellin,DATA,YEAR
2007/w01,1,"[2007, w01]",2007
2007/w02,0,"[2007, w02]",2007
2007/w03,0,"[2007, w03]",2007
2007/w04,0,"[2007, w04]",2007
2007/w05,0,"[2007, w05]",2007
...,...,...,...
2019/w48,15,"[2019, w48]",2019
2019/w49,20,"[2019, w49]",2019
2019/w50,30,"[2019, w50]",2019
2019/w51,14,"[2019, w51]",2019


In [9]:
# Convert to format year + week 
x_cases_transposed_medellin['YearWeek'] = x_cases_transposed_medellin['DATA'].apply(lambda x: x[0] + x[1].split('w')[1])  ##concate year and week index 0 and 1 in a list and apply to all rows
x_cases_transposed_medellin

,cases_medellin,DATA,YEAR,YearWeek
2007/w01,1,"[2007, w01]",2007,200701
2007/w02,0,"[2007, w02]",2007,200702
2007/w03,0,"[2007, w03]",2007,200703
2007/w04,0,"[2007, w04]",2007,200704
2007/w05,0,"[2007, w05]",2007,200705
...,...,...,...,...
2019/w48,15,"[2019, w48]",2019,201948
2019/w49,20,"[2019, w49]",2019,201949
2019/w50,30,"[2019, w50]",2019,201950
2019/w51,14,"[2019, w51]",2019,201951


In [10]:
#YearWeekInt --> changed to int
x_cases_transposed_medellin['YearWeek'] = x_cases_transposed_medellin['YearWeek'].astype(int)
x_cases_transposed_medellin.head()  #integer

,cases_medellin,DATA,YEAR,YearWeek
2007/w01,1,"[2007, w01]",2007,200701
2007/w02,0,"[2007, w02]",2007,200702
2007/w03,0,"[2007, w03]",2007,200703
2007/w04,0,"[2007, w04]",2007,200704
2007/w05,0,"[2007, w05]",2007,200705


In [11]:
x_cases_transposed_medellin['LastDayWeek'] = pd.to_datetime((x_cases_transposed_medellin['YearWeek']-1).astype(str) + "6" , format="%Y%U%w")   # 200701 + 6 -> yearweeknumber + Lastday of week 
##-1 -> our data weeknumber e.g. Yearweek 200701 yearweek starts from 01 as week 1 and datatime library starts from 00 as week 1 -> 1 -52 weeks data -> 0-51 weeks datelibrary
## %Uweek number 0 51 = 1 52  
##%w last day of week    This is taken from epiweek (1st epi week of 2007 starts 31-Dec-2016 ends 6-Jan-2017) 
##Library takes in yearweeknumber + weekday  -> the date form the last day of the week 
x_cases_transposed_medellin

,cases_medellin,DATA,YEAR,YearWeek,LastDayWeek
2007/w01,1,"[2007, w01]",2007,200701,2007-01-06
2007/w02,0,"[2007, w02]",2007,200702,2007-01-13
2007/w03,0,"[2007, w03]",2007,200703,2007-01-20
2007/w04,0,"[2007, w04]",2007,200704,2007-01-27
2007/w05,0,"[2007, w05]",2007,200705,2007-02-03
...,...,...,...,...,...
2019/w48,15,"[2019, w48]",2019,201948,2019-11-30
2019/w49,20,"[2019, w49]",2019,201949,2019-12-07
2019/w50,30,"[2019, w50]",2019,201950,2019-12-14
2019/w51,14,"[2019, w51]",2019,201951,2019-12-21


In [12]:
# Take month to append and duplicate temperature and precipitation
x_cases_transposed_medellin['MONTH'] = pd.DatetimeIndex(x_cases_transposed_medellin['LastDayWeek']).month  
x_cases_transposed_medellin    ##w5 Jan 2days , Feb 5 days --> MonthMax --> the month for the last day of the week --> Feb wins!

,cases_medellin,DATA,YEAR,YearWeek,LastDayWeek,MONTH
2007/w01,1,"[2007, w01]",2007,200701,2007-01-06,1
2007/w02,0,"[2007, w02]",2007,200702,2007-01-13,1
2007/w03,0,"[2007, w03]",2007,200703,2007-01-20,1
2007/w04,0,"[2007, w04]",2007,200704,2007-01-27,1
2007/w05,0,"[2007, w05]",2007,200705,2007-02-03,2
...,...,...,...,...,...,...
2019/w48,15,"[2019, w48]",2019,201948,2019-11-30,11
2019/w49,20,"[2019, w49]",2019,201949,2019-12-07,12
2019/w50,30,"[2019, w50]",2019,201950,2019-12-14,12
2019/w51,14,"[2019, w51]",2019,201951,2019-12-21,12


### JOIN TEMPERATURE AND CASES AND PRECIPITATION

In [13]:
merge_cases_temp = x_cases_transposed_medellin.merge(temperature_medellin, on=['LastDayWeek'], how='inner')
merge_cases_temp

,cases_medellin,DATA,YEAR,YearWeek,LastDayWeek,MONTH,temperature
0,1,"[2007, w01]",2007,200701,2007-01-06,1,19.615357
1,0,"[2007, w02]",2007,200702,2007-01-13,1,22.234000
2,0,"[2007, w03]",2007,200703,2007-01-20,1,22.268857
3,0,"[2007, w04]",2007,200704,2007-01-27,1,23.546000
4,0,"[2007, w05]",2007,200705,2007-02-03,2,25.353071
...,...,...,...,...,...,...,...
671,15,"[2019, w48]",2019,201948,2019-11-30,11,21.430286
672,20,"[2019, w49]",2019,201949,2019-12-07,12,23.005214
673,30,"[2019, w50]",2019,201950,2019-12-14,12,23.116214
674,14,"[2019, w51]",2019,201951,2019-12-21,12,21.851143


In [14]:
# Marge and duplicate precipitaton data
merge_cases_temp_precip = merge_cases_temp.merge(precipitation_medellin, on=['LastDayWeek'], how='inner')
merge_cases_temp_precip

,cases_medellin,DATA,YEAR,YearWeek,LastDayWeek,MONTH,temperature,percipitation_medellin,percipitation_medellin_rural,percipitation_medellin_urbano
0,1,"[2007, w01]",2007,200701,2007-01-06,1,19.615357,10.400423,11.932660,9.624380
1,0,"[2007, w02]",2007,200702,2007-01-13,1,22.234000,19.510615,20.182476,19.153142
2,0,"[2007, w03]",2007,200703,2007-01-20,1,22.268857,2.036514,0.825607,2.551441
3,0,"[2007, w04]",2007,200704,2007-01-27,1,23.546000,19.405491,21.616968,18.439284
4,0,"[2007, w05]",2007,200705,2007-02-03,2,25.353071,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...
671,15,"[2019, w48]",2019,201948,2019-11-30,11,21.430286,15.607313,15.862694,15.462729
672,20,"[2019, w49]",2019,201949,2019-12-07,12,23.005214,9.035788,6.591552,10.151920
673,30,"[2019, w50]",2019,201950,2019-12-14,12,23.116214,26.450263,26.978917,26.226603
674,14,"[2019, w51]",2019,201951,2019-12-21,12,21.851143,13.327096,14.847955,12.606804


In [17]:
# Dataframe to CSV
merge_cases_temp_precip.to_csv(r'Data/merge_cases_temp_precip_medellin_2019.csv')